## Authenticate Google Earth Engine Account

In [1]:
import ee
# Initialize the GEE API
try:
    ee.Initialize()
except Exception as ee:
    ee.Authenticate()
    ee.Initialize()

## Main toolbox

In [ ]:
""" Import the libraries required by the program"""
#------------------------------------------------------------------------------------------------------
# geemap:A Python package for interactive mapping with Google Earth Engine, ipyleaflet, and ipywidgets
# Documentation: https://geemap.org
import geemap
# from geemap import ee_basemaps

import matplotlib.pyplot as plt

import numpy as np

#eemont
import eemont

# import developed utilities
import Utilities as ut

# geetols: Google earth engine tools
# https://github.com/gee-community/gee_tools
import geetools
from geetools import tools

# hydrafloods: Hydrologic Remote Sensing Analysis for Floods
#https://github.com/Servir-Mekong/hydra-floods
import hydrafloods as hf
from hydrafloods import geeutils

# Ipywidgets for GUI design
import ipywidgets as ipw
from IPython.display import display
from ipywidgets import HBox, VBox, Layout

# A simple Python file chooser widget for use in Jupyter/IPython in conjunction with ipywidgets
# https://pypi.org/project/ipyfilechooser/
from ipyfilechooser import FileChooser

# Plotly Python API for interactive graphing
import plotly
import plotly.express as px
import plotly.graph_objects as go

# Pandas -  Python Data Analysis Library for data analysis and manipulation
import pandas as pd

# Miscellaneous Python modules
from datetime import datetime, timedelta
import os

#----------------------------------------------------------------------------------------------------

""" UI Design"""
#---------------------------------------------------------------------------------------------------
# Program Title
Title_text =  ipw.HTML(
    "<h3 class= 'text-center'><font color = 'blue'>Python-GEE Surface Water Analyzer Toolbox v.1.0.0</font>")
style = {'description_width': 'initial'}


# Image Processing Tab
#************************************************************************************************
# Image Parameters UI
dataset_description = ipw.Label('Satellite Imagery Parameters')

dataset_Label = ipw.Label('Select Dataset:', layout=Layout(margin='5px 0 0 5px')) #top right bottom left

Platform_options = ['Landsat', 'Sentinel-1', 'Sentinel-2', 'USDA NAIP' ]

Platform_dropdown = ipw.Dropdown(options = Platform_options, value = None,
                                   layout=Layout(width='150px', margin='5px 0 0 5px'))

PlatformType = HBox([dataset_Label, Platform_dropdown])

# Study period definition
#************************************************************************************************
# Start date picker
lbl_start_date = ipw.Label('Start Date:', layout=Layout(margin='5px 0 0 5px'))

start_date = ipw.DatePicker(value = datetime.now()-timedelta(7), disabled=False, 
                            layout=Layout(width='150px', margin='5px 0 0 30px'))

start_date_box = HBox([lbl_start_date, start_date])

# End date picker
lbl_end_date = ipw.Label('End Date:', layout=Layout(margin='5px 0 0 5px'))

end_date = ipw.DatePicker(value = datetime.now(), disabled=False, 
                          layout=Layout(width='150px', margin='5px 0 0 34px'))

end_date_box = HBox([lbl_end_date, end_date])

datePickers = VBox([start_date_box, end_date_box])


# Cloud threshold for filtering data
#************************************************************************************************
# Set cloud threshold
cloud_threshold = ipw.IntSlider(description = 'Cloud Threshold:', orientation = 'horizontal',
                                 value = 50, step = 5, style = style)

imageParameters = VBox([dataset_description, PlatformType, datePickers, cloud_threshold], 
                   layout=Layout(width='305px', border='solid 2px black'))


# Study Area definition
#************************************************************************************************
# Option to use a map drawn boundary or upload shapefile
StudyArea_description = ipw.Label('Study Area Definition')

user_preference = ipw.RadioButtons(options=['Map drawn boundary','Upload boundary'], value='Map drawn boundary')

file_selector = FileChooser(description = 'Upload', filter_pattern = ["*.shp"], use_dir_icons = True)

# Retrieve and process satellite images
#***********************************************************************************************
# Button to retrieve and process satellite images from the GEE platform
imageProcessing_Button = ipw.Button(description = 'Process images',
                                    tooltip='Click to process images', button_style = 'info',
                                   layout=Layout(width='150px', margin='5px 0 0 50px', border='solid 2px black'))

# Study area UI and process button container
# ************************************************************************************************
StudyArea = VBox(children = [StudyArea_description, user_preference, imageProcessing_Button], 
                   layout=Layout(width='300px', border='solid 2px black', margin='0 0 0 10px'))



# Results UI for displaying number and list of files
#*****************************************************************************************************
lbl_results = ipw.Label('Processing Results')

lbl_images = ipw.Label('No. of processed images:')

lbl_RetrievedImages = ipw.Label()

display_no_images = HBox([lbl_images, lbl_RetrievedImages])

lbl_files = ipw.Label('List of files:')

lst_files = ipw.Select(layout=Layout(width='360px', height='100px'))

image_Results = VBox([lbl_results, display_no_images, lbl_files, lst_files], 
                   layout=Layout(width='400px', border='solid 2px black', margin='0 0 0 10px'))


# Container for Image Processing Tab
#************************************************************************************************
imageProcessing_tab = HBox([imageParameters, StudyArea, image_Results])


# Water Extraction Tab
#*************************************************************************************************
# Water extraction indices
water_index_options = ['NDWI','MNDWI','DSWE', 'AWEInsh', 'AWEIsh']

lbl_indices = ipw.Label('Water Index:', layout=Layout(margin='5px 0 0 5px'))

water_indices = ipw.Dropdown(options = water_index_options, value = 'NDWI',
                            layout=Layout(width='100px', margin='5px 0 0 63px'))

display_indices = HBox([lbl_indices, water_indices])

# Color widget for representing water
lbl_color = ipw.Label('Color:', layout=Layout(margin='5px 0 0 5px'))

index_color =  ipw.ColorPicker(concise = False, value = 'blue',layout=Layout(width='100px', margin='5px 0 0 101px'))

display_color_widget = HBox([lbl_color, index_color])

# Water index threshold selection
threshold_options = ['Simple','Otsu']

lbl_threshold_method = ipw.Label('Thresholding Method:', layout=Layout(margin='5px 0 0 5px'))

threshold_dropdown = ipw.Dropdown(options = threshold_options,value = 'Simple',
                            layout=Layout(width='100px', margin='5px 0 0 10px'))

display_thresholds = HBox([lbl_threshold_method, threshold_dropdown])


lbl_threshold = ipw.Label('Threshold value:', layout=Layout(margin='5px 0 5px 5px'))

threshold_value = ipw.BoundedFloatText(value=0.000, min = -1.0, max = 1.0, step = 0.050,
                                       layout=Layout(width='100px', margin='5px 0 0 40px'))

display_threshold_widget = HBox([lbl_threshold, threshold_value])

water_index_Box = VBox([display_indices, display_thresholds, display_threshold_widget, display_color_widget],
                      layout=Layout(width='250px', border='solid 2px black'))

extractWater_Button = ipw.Button(description = 'Extract Water', tooltip='Click to extract surface water', 
                                button_style = 'info', 
                                 layout=Layout(width='150px', margin='5px 0 0 20px', border='solid 2px black'))

Extraction_tab = HBox([water_index_Box, extractWater_Button])
extractWater_Button.disabled = True

# Spatial Analysis Tab
#**************************************************************************************************

water_Frequency_button = ipw.Button(description = 'Compute Water Frequency',
                                    tooltip='Click to compute water occurence frequency',
                                    button_style = 'info', 
                                    layout=Layout(width='200px', border='solid 2px black',margin='5 0 0 50px'))
water_Frequency_button.disabled = True

Depths_Button = ipw.Button(description = 'Compute Depth Map',
                                tooltip='Click to generate depth maps', button_style = 'info',
                                layout=Layout(width='200px', border='solid 2px black',margin='5 0 0 50px'))
Depths_Button.disabled = True

elevData_options = ipw.Dropdown(options=['NED','SRTM','User DEM'], value='NED', description='Elevation Dataset:',
                                layout=Layout(width='210px', margin='0 0 0 10px'), style = style)
elevData_options.disabled = True

userDEM = ipw.Dropdown(description='Enter GEE asset ID:', 
                  layout=Layout(width='500px', margin='0 0 0 10px'), style = style)

lbl_Elev = ipw.Label('Elevation Dataset:', layout=Layout(margin='0 0 0 10px'))

elev_Box = HBox([Depths_Button, elevData_options])

zonalAnalysis_Button = ipw.Button(description = 'Zonal Analysis',
                                tooltip='Click to remove clouds', button_style = 'info',
                               layout=Layout(width='200px', border='solid 2px black',margin='5 0 0 50px'))

# Spatial_Analysis_Tab = VBox([water_Frequency_button, elev_Box, zonalAnalysis_Button])
Spatial_Analysis_Tab = VBox([water_Frequency_button, elev_Box])


# Ploting and Statistics Tab
#***************************************************************************************************
lbl_Area_Plotting = ipw.Label(value ='Surface Water Area Computation:')
area_unit = ipw.Dropdown(options = ['Square m','Square Km', 'Hectares', 'Acre'], value = 'Square m',
                            description = 'Unit for water surface area:', style=style,
                            tooltip='Select unit for areas')

plot_button = ipw.Button(description = 'Compute and Plot Areas', tooltip='Click to plot graph', button_style = 'info',
                        layout=Layout(width='170px', margin='10 0 0 100px', border='solid 2px black'))
plot_button.disabled = True

lbl_depth_Plotting = ipw.Label(value ='Plot depth at a location:', layout=Layout(margin='10px 0 0 0'))

point_preference = ipw.RadioButtons(options=['Map drawn point','Enter coordinates'], 
                                    value='Map drawn point')

coordinates_textbox = ipw.Text(layout=Layout(width='200px'))
lbl_coordinates = ipw.Label(value='Enter Long, Lat in decimal degrees')

# point_selector = FileChooser(description = 'Upload point', filter_pattern = ["*.shp"], use_dir_icons = True)

depth_plot_button = ipw.Button(description = 'Plot depths', tooltip='Click to plot depth hydrograph', button_style = 'info',
                        layout=Layout(width='170px', margin='10 0 0 100px', border='solid 2px black'))
depth_plot_button.disabled = True

depth_box = VBox(children = [lbl_depth_Plotting,point_preference, depth_plot_button])

plotting_box = VBox([lbl_Area_Plotting, area_unit, plot_button, depth_box], 
                    layout=Layout(width='310px', border='solid 2px black'))


file_selector1 = FileChooser(description = 'Select folder and filename', filter_pattern = "*.csv", use_dir_icons = True)

file_selector1.title = '<b>Select Folder and Filename</b>'

file_selector1.default_path = os.getcwd()

save_data_button = ipw.Button(description = 'Save Data',tooltip='Click to save computed areas to file',button_style = 'info',
                               layout=Layout(width='100px', border='solid 2px black',margin='5 0 0 50px'))

save_box = VBox(children=[file_selector1, save_data_button],
               layout=Layout(width='550px', border='solid 2px black', margin='0 0 0 10px'))

plot_stats_tab = HBox(children=[plotting_box, save_box])

# Downloads Tab
#***************************************************************************************************
files_to_download = ipw.RadioButtons(options=['Satellite Images', 'Water Mask', 'Water Frequency', 'Depth Maps',
                                              'DSWE Images'], value='Satellite Images', 
                                     description='Files to download:', style = style)

download_location = ipw.RadioButtons(options=['Google Drive', 'Local Disk'], 
                                     value='Google Drive', description='Download Location:', style = style)

folder_name = ipw.Text(description='Folder Name:')

folder_selector = FileChooser(description = 'Select Folder', show_only_dirs = True, use_dir_icons = True)

folder_selector.title = '<b>Select a folder</b>'

folder_selector.default_path = os.getcwd()

download_button = ipw.Button(description = 'Download',
                                    tooltip='Click to plot download water images', button_style = 'info')


download_settings = VBox(children=[files_to_download, download_location, folder_name])

download_tab = HBox([download_settings, download_button])

# Functions to control UI changes and parameter settings
#****************************************************************************************************
def platform_index_change(change):
    """
    Function to set image visualization parameters, hide or show some UI componets and
    show water indices that are applicable to the type of satellite image selected
        
    args:
        None

    returns:
        None
    """
    try:
        global img_type
        global visParams
        global water_index_options
        
        if Platform_dropdown.value == 'Landsat':
            visParams = {'bands': ['red', 'green', 'blue'],
                  'min': 0,
                  'max': 3000,
                  'gamma': [0.95, 1.1, 1]}
            cloud_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            threshold_value.disabled = False
            water_indices.options = ['NDWI','MNDWI','DSWE','AWEInsh', 'AWEIsh']
            threshold_dropdown.options = ['Simple','Otsu']
            
        elif Platform_dropdown.value == 'Sentinel-1':
            visParams = {'min': -25,'max': 5}
            cloud_threshold.disabled = True
            water_indices.disabled = True
            index_color.disabled = True
            threshold_value.disabled = True
            threshold_dropdown.options = ['Otsu']
            
        elif Platform_dropdown.value == 'Sentinel-2':
            visParams = {'bands': ['red', 'green', 'blue'],
              'min': 0.0,
              'max': 3000}
            cloud_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            threshold_value.disabled = False
            water_indices.options = ['NDWI','MNDWI','AWEInsh', 'AWEIsh']
            threshold_dropdown.options = ['Simple','Otsu']
            
        elif Platform_dropdown.value == 'USDA NAIP':
            visParams = {'bands': ['R', 'G','B'],
                        'min': 0.0,
                        'max': 255.0}
            threshold_value.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            water_indices.options = ['NDWI']
            threshold_dropdown.options = ['Simple','Otsu']
    except Exception as e:
             print(e)

# Link widget to function
Platform_dropdown.observe(platform_index_change, 'value')

def showFileSelector(button):
    """
    Function to show or hide shapefile upload widget
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        StudyArea.children = [StudyArea_description, user_preference, file_selector, imageProcessing_Button]
    else:
        StudyArea.children = [StudyArea_description, user_preference,imageProcessing_Button]

# Link widget to file selector function
user_preference.observe(showFileSelector, names='index')

def showLocationSelector(button):
    """
    Function to show or hide folder selector
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        download_settings.children = [files_to_download, download_location, folder_selector]
    else:
        download_settings.children = [files_to_download, download_location, folder_name]
        
# Link widget to folder selector function
download_location.observe(showLocationSelector, names='index')

def pointOptions_selector(button):
    """
    Function to show or hide folder selector
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        depth_box.children = [lbl_depth_Plotting,point_preference, lbl_coordinates, coordinates_textbox, 
                              depth_plot_button]
    else:
        depth_box.children = [lbl_depth_Plotting,point_preference, depth_plot_button]

        
# Link widget to folder selector function
point_preference.observe(pointOptions_selector, names='index')


def thresholdSelection(change):
    if threshold_dropdown.value =='Otsu':
        threshold_value.disabled = True
    else:
        threshold_value.disabled = False
        
# Link widget to threshold method selection function
threshold_dropdown.observe(thresholdSelection, 'value')

def demSelection(change):
    if elevData_options.value == 'User DEM':
        folder = ee.data.getAssetRoots()[0]['id']
        assets = ee.data.listAssets({'parent':folder})
        # filter only image assets
        filtered_asset = list(filter(lambda asset: asset['type'] == 'IMAGE', assets['assets']))
        # create a list of image assets
        list_assets = [sub['id'] for sub in filtered_asset]
        elev_Box.children = [Depths_Button, elevData_options, userDEM]
        userDEM.options = list_assets # set dropdown options to list of image assets
    else:
        elev_Box.children = [Depths_Button, elevData_options]

# Link widget to function
elevData_options.observe(demSelection, 'value')

#****************************************************************************************************

# Full UI
#***************************************************************************************************
tab_children = [imageProcessing_tab, Extraction_tab, Spatial_Analysis_Tab, plot_stats_tab, download_tab]

tab = ipw.Tab()
tab.children = tab_children

# changing the title of the first and second window
tab.set_title(0, 'Image Processing')
tab.set_title(1, 'Water Extraction')
tab.set_title(2, 'Spatial Analysis')
tab.set_title(3, 'Plotting & Stats')
tab.set_title(4, 'Download & Export')


# Plotting outputs and feedback to user
#***************************************************************************************************

feedback = ipw.Output()
OUTPUTS = VBox([feedback])

# create map instance
Map =  geemap.Map()
Map.add_basemap('HYBRID')

GUI = VBox([Title_text,tab,Map])

def reset_map(self):
    Map.clear_layers()
    Map.remove_control(Map.colorbar)
    Map.add_basemap('HYBRID')

# Function to clip images
def clipImages(img):
    """
    Function to clip images

    args:
        Image

    returns:
        Clipped image
    """
    orig = img
    clipped_image = img.clip(site).copyProperties(orig, orig.propertyNames())
    return clipped_image    


def process_images(self):
    """
    Function to retrieve and process satellite images from GEE platform
        
    args:
        None

    returns:
        None
    """
    try:
        global filtered_Collection
        global filtered_landsat
        global clipped_images
        global imageType
        global dates
        global site
        global img_scale
        global file_list
        global StartDate
        global EndDate

        fig.data = [] # clear existing plot
        # reset the map
#             Map.clear_layers()
#             Map.add_basemap('HYBRID')
#             Map.remove_control(Map.colorbar)

        lbl_RetrievedImages.value = 'Processing....'
        cloud_thresh = cloud_threshold.value

        # Define study area based on user preference
        if user_preference.index == 1:
            file = file_selector.selected  
            site = ut.load_boundary(file)
            Map.addLayer(site, {}, 'AOI')
            Map.center_object(site, 15)
        else:
            site = ee.FeatureCollection(Map.draw_last_feature)

        # get widget values
        imageType = Platform_dropdown.value
        StartDate = ee.Date.fromYMD(start_date.value.year,start_date.value.month,start_date.value.day)
        EndDate = ee.Date.fromYMD(end_date.value.year,end_date.value.month,end_date.value.day)

        # filter image collection based on date, study area and cloud threshold(depends of datatype)
        if imageType == 'Landsat':
            filtered_landsat = ut.load_Landsat(site, StartDate, EndDate, cloud_thresh)
            filtered_Collection = filtered_landsat.map(ut.maskLandsatclouds)
        elif imageType == 'Sentinel-2':
            Collection_before = ut.load_Sentinel2(site, StartDate, EndDate, cloud_thresh)
            filtered_Collection = Collection_before.map(ut.maskS2clouds)
        elif imageType == 'Sentinel-1':
            Collection_before = ut.load_Sentinel1(site, StartDate, EndDate)
            # apply speckle filter algorithm or smoothing
            filtered_Collection = Collection_before.map(hf.lee_sigma)
        elif imageType == 'USDA NAIP':
            filtered_Collection = ut.load_NAIP(site, StartDate, EndDate)

        # Clip images to study area
        clipped_images = filtered_Collection.map(clipImages)

        # Mosaic same day images
        clipped_images = tools.imagecollection.mosaicSameDay(clipped_images)

        # Add first image in collection to Map
        first_image = clipped_images.first()
        if imageType == 'Sentinel-1':
            img_scale = first_image.select(0).projection().nominalScale().getInfo()
        else:
            bandNames = first_image.bandNames().getInfo()
            img_scale = first_image.select(str(bandNames[0])).projection().nominalScale().getInfo()

        Map.addLayer(clipped_images.first(), visParams, imageType)

        # Get no. of processed images
        no_of_images = filtered_Collection.size().getInfo()

        # Display number of images
        lbl_RetrievedImages.value = str(no_of_images)

        # List of files
        file_list = filtered_Collection.aggregate_array('system:id').getInfo()
        # display list of files
        lst_files.options = file_list
        extractWater_Button.disabled = False # enable the water extraction button

    except Exception as e:
             print(e)
             print('An error occurred during processing.')
                    

def Water_Extraction(self):
    """
    Function to extract surface water from satellite images

    args:
        None

    returns:
        None
    """
    try:
        global water_images
        global dswe_images
        global waterMasks

        color_palette = index_color.value       
        # Function to extract water using NDWI or MNDWI from multispectral images
        def extract_water(img):
            """
            Function to extract surface water from Landsat and Sentinel-2 images using
            water extraction indices: NDWI, MNDWI, and AWEI

            args:
                Image

            returns:
                Image with water mask
            """
            index_image = ee.Image(1)
            if water_indices.value == 'NDWI':
                if imageType == 'Landsat' or imageType == 'Sentinel-2':
                    bands = ['green', 'nir']
                elif imageType == 'USDA NAIP':
                    bands = ['G', 'N']
                index_image = img.normalizedDifference(bands).rename('waterMask')

            elif water_indices.value == 'MNDWI':
                if imageType == 'Landsat':
                    bands = ['green', 'swir1']
                    index_image = img.normalizedDifference(bands).rename('waterMask')

                elif imageType == 'Sentinel-2':
                    # Resample the swir bands from 20m to 10m
                    resampling_bands = img.select(['swir1','swir2'])
                    img = img.resample('bilinear').reproject(**
                              {'crs': resampling_bands.projection().crs(),
                                'scale':10
                              })
                    bands = ['green', 'swir1']
                    index_image = img.normalizedDifference(bands).rename('waterMask')

            elif water_indices.value == 'AWEInsh':
                index_image = img.expression(
                        '(4 * (GREEN - SWIR1)) - ((0.25 * NIR)+(2.75 * SWIR2))', {
                            'NIR': img.select('nir'),
                            'GREEN': img.select('green'),
                            'SWIR1': img.select('swir1'),
                            'SWIR2': img.select('swir2')
                        }).rename('waterMask')

            elif water_indices.value == 'AWEIsh':
                index_image = img.expression(
                        '(BLUE + (2.5 * GREEN) - (1.5 * (NIR + SWIR1)) - (0.25 * SWIR2))', {
                            'BLUE':img.select('blue'),
                            'NIR': img.select('nir'),
                            'GREEN': img.select('green'),
                            'SWIR1': img.select('swir1'),
                            'SWIR2': img.select('swir2')
                        }).rename('waterMask')

            # Compute threshold
            if threshold_dropdown.value == 'Simple': # Simple value no dynamic thresholding
                nd_threshold = threshold_value.value
                water_image = index_image.gt(nd_threshold).copyProperties(img, ['system:time_start'])
            elif threshold_dropdown.value == 'Otsu':
                reducers = ee.Reducer.histogram(255,2).combine(reducer2=ee.Reducer.mean(), sharedInputs=True)\
                    .combine(reducer2=ee.Reducer.variance(), sharedInputs= True)
                histogram = img.select('nir').reduceRegion(
                                reducer=reducers,
                                geometry=site.geometry(),
                                scale=img_scale,
                                bestEffort=True)
                nd_threshold = ut.otsu(histogram.get('nir_histogram')) # get threshold from the nir band

                water_image = img.select('nir').lt(nd_threshold).copyProperties(img, ['system:time_start'])    
            return water_image

        # Function to extract water from SAR Sentinel 1 images
        def add_S1_waterMask(img):
            """
            Function to extract surface water from Sentinel-1 images Otsu algorithm

            args:
                Image

            returns:
                Image with water mask
            """
            # Compute histogram
            reducers = ee.Reducer.histogram(255,2).combine(reducer2=ee.Reducer.mean(), sharedInputs=True)\
                .combine(reducer2=ee.Reducer.variance(), sharedInputs= True)
            histogram = img.select('VV').reduceRegion(
            reducer=reducers,
            geometry=site.geometry(),
            scale=img_scale,
            bestEffort=True)

            # Calculate threshold via function otsu (see before)
            threshold = ut.otsu(histogram.get('VV_histogram'))

            # get watermask
            waterMask = img.select('VV').lt(threshold).rename('waterMask')
    #             waterMask = waterMask.updateMask(waterMask) #Remove all pixels equal to 0
            return img.addBands(waterMask)

        def maskDSWE_Water(img):
            waterImage = img.select('dswe').rename('waterMask')
            watermask = waterImage.gt(0).And(waterImage.lt(5)).copyProperties(img, ['system:time_start'])
            return watermask
        def mask_Water(img):
            return img.select('waterMask').selfMask().copyProperties(img, ['system:time_start'])

        if imageType == 'Sentinel-1':
            water_images = clipped_images.map(add_S1_waterMask).select('waterMask')
            waterMasks = water_images.map(mask_Water)
            visParams = {'min': 0,'max': 1, 'palette': color_palette}
            Map.addLayer(waterMasks.first(), visParams, 'Water')
        elif imageType == 'Landsat':
            if water_indices.value == 'DSWE':
                dem = ee.Image('USGS/NED')
                dswe_images = ut.DSWE(filtered_landsat, dem, site)
                 # Viz parameters: classes: 0, 1, 2, 3, 4, 9
                dswe_viz = {'min':0, 'max': 9, 'palette': ['000000', '002ba1', '6287ec', '77b800', 'c1bdb6', '000000', 
                                                           '000000', '000000', '000000', 'ffffff']}
                water_images = dswe_images.map(maskDSWE_Water)
                waterMasks = water_images.map(mask_Water)
                Map.addLayer(dswe_images.first(), dswe_viz, 'DSWE')
            else:
                water_images = clipped_images.map(extract_water)
                waterMasks = water_images.map(mask_Water)
            Map.addLayer(waterMasks.first(), {'palette': color_palette}, 'Water')

        else:
            water_images = clipped_images.map(extract_water)
            waterMasks = water_images.map(mask_Water)
            Map.addLayer(waterMasks.first(), {'palette': color_palette}, 'Water')
            
        water_Frequency_button.disabled = False
        Depths_Button.disabled = False
        elevData_options.disabled = False
        plot_button.disabled = False
        
    except Exception as e:
             print(e)
             print('An error occurred during computation.')

# threshold_value.observe(Water_Extraction, 'value')
                
def calc_area(img):
    """
    Function to calculate area of water pixels

    args:
        Water mask image

    returns:
        Water image with calculated total area of water pixels
    """
    global unit_symbol
    unit = area_unit.value
    divisor = 1
    if unit =='Square Km':
        divisor = 1e6
        unit_symbol = 'Square km'
    elif unit =='Hectares':
        divisor = 1e4
        unit_symbol = 'Ha'
    elif unit =='Square m':
        divisor = 1
        unit_symbol = 'Sqaure m'
    else:
        divisor = 4047
        unit_symbol = 'acre'
        
    pixel_area = img.multiply(ee.Image.pixelArea()).divide(divisor)
    img_area = pixel_area.reduceRegion(**{
                        'geometry': site.geometry(),
                        'reducer': ee.Reducer.sum(),
                        'scale': img_scale,
                        'maxPixels': 1e13
                        })
    
    return img.set({'water_area': img_area})


def plot_areas(self):
    """
    Function to plot a time series of calculated water area for each water image
    and to cycle through 
    
    args:
        None

    returns:
        None
    """
    try:
        global df
        # Compute water areas
        water_areas = waterMasks.map(calc_area)
        water_stats = water_areas.aggregate_array('water_area').getInfo()

        dates = waterMasks.aggregate_array('system:time_start')\
            .map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()
        
        dates_lst = [datetime.strptime(i, '%Y-%m-%d') for i in dates]
        y = [item.get('waterMask') for item in water_stats]
        df = pd.DataFrame(list(zip(dates_lst,y)), columns=['Date','Area'])
        
        fig.data = []

        fig.add_trace(go.Scatter(x=df['Date'], y=df['Area'], name='Water Hydrograph', 
                mode='lines+markers', line=dict(dash = 'solid', color ='Blue', width = 0.5)))

        fig.layout.title = '<b>Surface Water Area Hydrograph<b>'
        fig.layout.titlefont = dict(family="Arial",size=24)
        fig.layout.title.x = 0.5
        fig.layout.title.y = 0.9

        scatter = fig.data[0] # set figure data to scatter for click function

        color_palette = index_color.value 

        # Function to select and show water image on clicking the graph
        def update_point(trace, points, selector):
            global wImage
            global selected_sat
            date = df['Date'].iloc[points.point_inds].values[0]
            date = pd.to_datetime(str(date))
            wImage = waterMasks.closest(date).first()
            selected_sat = clipped_images.closest(date).first()
            Map.addLayer(wImage, {'palette': color_palette}, 'Water')

        scatter.on_click(update_point)
            
    except Exception as e:
            print(e)
            print('An error occurred during computation.')

def save_data(self):
    filename = file_selector1.selected
    df.to_csv(filename, index=False)

def dowload_images(self):
#         with feedback:
        try:
            path = folder_selector.selected_path
            folder = folder_name.value
            name_Pattern = '{sat}_{system_date}_{imgType}'
            date_pattern = 'ddMMMy'
            extra = dict(sat=imageType, imgType = 'Water')
            if files_to_download.index == 0:
                download_images = clipped_images
                extra = dict(sat=imageType, imgType = 'Satellite')
            elif files_to_download.index == 1:
                download_images = water_images
                extra = dict(sat=imageType, imgType = 'Water')
            elif files_to_download.index == 2:
                download_images = ee.ImageCollection([water_occurence])
                name_Pattern = '{sat}_{start}_{end}_{imgType}'
                extra = dict(sat=imageType, imgType = 'Frequency', start=start_date.value.strftime("%x"),
                             end=end_date.value.strftime("%x"))
            elif files_to_download.index == 3:
                download_images = depth_maps
                extra = dict(sat=imageType, imgType = 'Depth')
            else:
                download_images = dswe_images
                extra = dict(sat=imageType, imgType = 'DSWE')

            if download_location.index == 0:
                task = geetools.batch.Export.imagecollection.toDrive(
                    collection = download_images,
                    folder = folder,
                    region = site.geometry(),
                    namePattern = name_Pattern,
                    scale = img_scale,
                    datePattern=date_pattern,
                    extra = extra,
                    verbose=True,
                    maxPixels = int(1e13))
                task
            else:
                geemap.ee_export_image_collection(download_images, out_dir=path)

            feedback.clear_output()

        except Exception as e:
                print(e)
                print('Download could not be completed')
                    
def water_frequency(self):
#     with feedback:
    try:
        global water_frequency
        global water_occurence
        water_occurence =  water_images.reduce(ee.Reducer.sum())
        water_frequency = water_occurence.divide(water_images.size()).multiply(100)
        visParams = {'min':0, 'max':100, 'palette': ['orange','yellow','blue','darkblue']}
        Map.addLayer(water_frequency, visParams, 'Water Frequency')

        colors = visParams['palette']
        vmin = visParams['min']
        vmax = visParams['max']

        Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="Water Frequency")
    except Exception as e:
                print(e)
                print('Frequency computation could not be completed')

def get_dates(col):
    dates = ee.List(col.toList(col.size()).map(lambda img: ee.Image(img).date().format()))
    return dates

# Function to count water pixels for each image   
def CountWaterPixels(img):
    count = img.reduceRegion(ee.Reducer.sum(), site).values().get(0)
    return img.set({'pixel_count': count})

def calc_depths(self):
    try:
        global depth_maps
        global filtered_Water_Images
        global depthParams
        if elevData_options.value =='NED':
            demSource = 'USGS/NED'
            band = 'elevation'
        elif elevData_options.value =='SRTM':
            demSource = 'USGS/SRTMGL1_003'
            band = 'elevation'
        else:
            demSource = str(userDEM.value)
            band = 'b1'

    #     dem = ee.Image("users/collinsowusu09/GDM").select(['b1']).clip(site)
        dem = ee.Image(demSource).select(band).clip(site)

        # get water pixel count per image
        countImages = waterMasks.map(CountWaterPixels)

        # Filter out only images containing water pixels to avoid error in depth estimation
        filtered_Water_Images = countImages.filter(ee.Filter.gt('pixel_count', 0))

        def estimateDepths(img):
            flood = img
            dem_mask = dem.mask(flood)

            polys = flood.addBands(dem_mask).reduceToVectors(**{
                    'geometry':site,
                    'scale':img_scale,
                    'reducer':ee.Reducer.max(),
                    'eightConnected': False,
                    'geometryType':'polygon',
                    'crs': flood.projection()
                    })

            polys2 = dem.reduceRegions(polys, ee.Reducer.max())
            propNames = polys2.first().propertyNames();
            polys2 = polys2.select(propNames, ['max_elev','system:index', 'label'])

            properties = ['max_elev']
            maxImage = polys2.filter(ee.Filter.notNull(properties))\
                        .reduceToImage(**{'properties': properties, 'reducer': ee.Reducer.first()})

            Depths = maxImage.subtract(dem_mask).rename('Depth')
            DepthFilter = Depths.where(Depths.lt(0),0)
            return DepthFilter.copyProperties(flood, flood.propertyNames())

        depth_maps = filtered_Water_Images.map(estimateDepths)

        first_depth_map = depth_maps.first()
        maxVal = first_depth_map.reduceRegion(ee.Reducer.max(),site, img_scale).values().get(0).getInfo()

        depthParams = {'min':0, 'max':round(maxVal,1), 'palette': ['1400f7','00f4e8','f4f000','f40000','960424']}
        #['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
        Map.addLayer(depth_maps.first(), depthParams, 'Depth')
        colors = depthParams['palette']
        Map.add_colorbar_branca(colors=colors, vmin=0, vmax=round(maxVal,1), layer_name='Depth')
        depth_plot_button.disabled = False # enable depth plotting

    except Exception as e:
            print(e)
    
def zonal_analysis(self):
    pass
#     try:
#         global areaImages
#         global point_areas
#         point = ee.FeatureCollection(Map.draw_last_feature)

#         def addArea(feature):
#             return feature.set({'Area': feature.geometry().area(2)})
        

#         def estimate_zone(img):
#     #         orig = img
#             polygons = img.addBands(img).reduceToVectors(**{
#                     'geometry':site,
#                     'scale':img_scale,
#                     'reducer':ee.Reducer.mean(),
#                     'eightConnected': False,
#                     'geometryType':'polygon',
#                     'crs': img.projection()
#                     })

#             filtered_poly = polygons.filterBounds(point)
#             poly_with_Area = filtered_poly.map(addArea)
#             area = poly_with_Area.aggregate_array('Area').getInfo()
#             return area

#          # get water pixel count per image
#         countImages = waterMasks.map(CountWaterPixels)

#         # Filter out only images containing water pixels to avoid error in depth estimation
#         filtered_Water_Images = countImages.filter(ee.Filter.gt('pixel_count', 0))

#         Areas = filtered_Water_Images.map(estimate_zone)
        
#         arealist = []
#         for x in Areas:
#             if not x:
#                 arealist.append(0)
#             else:
#                 arealist.append(x[0])
#                 return areas
        
#         print(arealist)
#     except Exception as e:
#         print(e)

def plot_depths(self):
    try: 
        global depths_df
        if point_preference.index == 0:
            point = ee.FeatureCollection(Map.draw_last_feature)
        else:
            coordinates = coordinates_textbox.value
            xy = coordinates.split(',')
            floated_xy = [float(i) for i in xy]
            point = ee.Geometry.Point(floated_xy)
            Map.addLayer(point, {}, 'Depth Point')

        ts_1 = depth_maps.getTimeSeriesByRegion(geometry = point,
                                  bands = ['Depth'],
                                  reducer = [ee.Reducer.mean()],
                                  scale = img_scale)

        depths_df = geemap.ee_to_pandas(ts_1)
        depths_df[depths_df == -9999] = np.nan
        depths_df = depths_df.fillna(0)
        depths_df['date'] = pd.to_datetime(depths_df['date'],infer_datetime_format = True)

        fig.data = []

        fig.add_trace(go.Scatter(x=depths_df['date'], y=depths_df['Depth'], name='Depth Hydrograph', 
                    mode='lines+markers', line=dict(dash = 'solid', color ='Red', width = 0.5)))

        fig.layout.yaxis.title = '<b>Depth (m)<b>'
        fig.layout.title = '<b>Water Depth Hydrograph<b>'
        fig.layout.titlefont = dict(family="Arial",size=24)
        fig.layout.title.x = 0.5
        fig.layout.title.y = 0.9
        
        scatter = fig.data[0] # set figure data to scatter for click function

        color_palette = index_color.value 

        # Function to select and show water image on clicking the graph
        def update_point(trace, points, selector):
            global wImage
            global selected_sat
            date = depths_df['date'].iloc[points.point_inds].values[0]
            date = pd.to_datetime(str(date))
            depthImage = depth_maps.closest(date).first()
#             selected_sat = clipped_images.closest(date).first()
            Map.addLayer(depthImage, depthParams, 'Depth')

        scatter.on_click(update_point)

    except Exception as e:
        print(e)

imageProcessing_Button.on_click(process_images)
extractWater_Button.on_click(Water_Extraction)
Depths_Button.on_click(calc_depths)
# zonalAnalysis_Button.on_click(zonal_analysis)
plot_button.on_click(plot_areas)
download_button.on_click(dowload_images)
water_Frequency_button.on_click(water_frequency)
save_data_button.on_click(save_data)
depth_plot_button.on_click(plot_depths)

# Display GUI widgets
display(GUI)
display(OUTPUTS)

# Create a plotly figure instance
fig = go.FigureWidget()
fig.update_layout(title = '<b>Surface Water Area Hydrograph<b>', 
                              title_x = 0.5, title_y = 0.90, title_font=dict(family="Arial",size=24),
                              template = "plotly_white",
                              xaxis =dict(title ='<b>Date<b>', linecolor = 'Black'),
                              yaxis=dict(title='Area (sqkm)', linecolor = 'Black'),
                              font_family="Arial")
# display plotly figure
fig
